# Анализ исходных данных

## 1 Общее описание набора данных

## 2 Анализ

### 2.1 Импорт библиотек и загрузка данных

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [5]:
BASE_DIR = os.path.dirname(os.path.dirname(os.getcwd()))

In [6]:
DATA_PATH_PREPARE = os.path.join(BASE_DIR, 'data', 'prepare', 'data_structed.xlsx')
DOMAIN_PATH = os.path.join(BASE_DIR, 'data', 'prepare', 'domain.xlsx')
PHYLUM_PATH = os.path.join(BASE_DIR, 'data', 'prepare', 'phylum.xlsx')
CLASS_PATH = os.path.join(BASE_DIR, 'data', 'prepare', 'class.xlsx')
ORDER_PATH = os.path.join(BASE_DIR, 'data', 'prepare', 'order.xlsx')
FAMILY_PATH = os.path.join(BASE_DIR, 'data', 'prepare', 'family.xlsx')
SPECIES_PATH = os.path.join(BASE_DIR, 'data', 'prepare', 'species.xlsx')

In [7]:
df_structed = pd.read_excel(DATA_PATH_PREPARE)
df_domain = pd.read_excel(DOMAIN_PATH)
df_phylum = pd.read_excel(PHYLUM_PATH)
df_class = pd.read_excel(CLASS_PATH)
df_order = pd.read_excel(ORDER_PATH)
df_family = pd.read_excel(FAMILY_PATH)
df_species = pd.read_excel(SPECIES_PATH)

In [71]:
LIST_FEATURES = list(df_species['name'].values)

In [8]:
df_structed.head()

,type_of_bacterium,count,target,group
0,Galactobacter,0,ПР,1
1,Corynebacterium,0,ПР,1
2,Adlercreutzia,5,ПР,1
3,Gordonibacter,0,ПР,1
4,Paraeggerthella,0,ПР,1


### 2.2 Проверка и обработка исходных данных

Проверяем по количеству в данных колонке `target` найдем общее количество признаков

In [14]:
df_species.head()

,id,id_family,name
0,1,1,Arthrobacter
1,2,1,Galactobacter
2,3,2,Corynebacterium
3,4,3,Adlercreutzia
4,5,3,Gordonibacter


In [15]:
len(df_species)

108

In [16]:
df_structed_grouped = df_structed.groupby(by=['target', 'group'])
df_structed_grouped.count()

type_of_bacterium  count
target      group                          
9495        1                    108    108
            2                    108    108
Fe          1                    108    108
            2                    108    108
АУ          1                    108    108
            2                    108    108
Биф         1                    108    108
            2                    108    108
Лакт        1                    108    108
            2                    108    108
Медь        1                    108    108
            2                    108    108
ПДР         1                    108    108
            2                    108    108
ПР          1                    107    107
            2                    108    108
Спор        1                    108    108
            2                    108    108
Хит         1                    108    108
            2                    108    108
Целлюлоза   1                    108    108
            2                    108    108
Энтеросгель 1                    108    108
            2                    108    108

In [17]:
df_structed_grouped.count()['count'][lambda x: x != 108]

target  group
ПР      1        107
Name: count, dtype: int64

Видим что для `ПР-1` не хватает одного показателя, можем его найти

In [18]:
df1 = df_structed[(df_structed['target'] == 'ПР') & (df_structed['group'] == 1)]
df2 = df_structed[(df_structed['target'] == 'ПР') & (df_structed['group'] == 2)]

In [19]:
set(df1.type_of_bacterium) ^ set(df2.type_of_bacterium)

{'Arthrobacter'}

В итоге для `ПР-1` не хвататет `'Arthrobacter'`

In [56]:
def make_transpose_by_col(df, ind_cols, feature_col, feature_col_available, col_values):
    """
    Разорачивание DataFrame по столбцу с индексами и признаками

    Args:
        df (pd.DataFrame): исходный датафрейм
        ind_cols (list): список столбцов в котором индекс (по каким столбцам разворачивать)
        feature_col (str): столбец признак
        feature_col_available (list): список допустимых значений по столбцу
        col_values (str): столбец значение
    """

    df_index = df[ind_cols].drop_duplicates().reset_index(drop=True)
    df_result_list = []
    for ind, row in df_index.iterrows():
        df_result_iter = []
        df_filter = df
        for col in ind_cols:
            df_filter = df_filter[df_filter[col] == row[col]]
            df_result_iter.append(row[col])
        df_filter = df_filter[[feature_col, col_values]]
        for col in feature_col_available:
            df_filter_col = df_filter[df_filter[feature_col] == col].reset_index(drop=True)
            if len(df_filter_col) != 1:
                df_result_iter.append(np.nan)
            else:
                df_result_iter.append(df_filter_col.loc[0, col_values])
        df_result_list.append(df_result_iter)

    df_columns_names = ind_cols + feature_col_available
    return pd.DataFrame(df_result_list, columns=df_columns_names)


In [ ]:
df_structed_transpose = make_transpose_by_col(df_structed, ['target', 'group'], 'type_of_bacterium', LIST_FEATURES, 'count')

In [64]:
df_structed_transpose.head()

,target,group,Arthrobacter,Galactobacter,Corynebacterium,Adlercreutzia,Gordonibacter,Paraeggerthella,Rubneribacter,Slackia,...,Turicimonas,Vampirovibrio,Enterobacter,Escherichia/Shigella,Klebsiella,unclassified_Enterobacteriaceae,Acinetobacter,Stenotrophomonas,unclassified_Bacteria,unclassified_Dothideomycetes
0,ПР,1,NaN,0,0,5,0,0,77,0,...,3,23,487,0,5,0,3,3,3,0
1,ПР,2,0.0,0,5,10,0,0,95,3,...,0,8,100,0,0,0,0,3,4,0
2,ПДР,1,0.0,0,3,0,0,0,22,0,...,0,0,56,6,0,0,0,4,0,0
3,ПДР,2,10.0,0,0,0,0,0,23,0,...,9,21,316,11,5,0,0,26,3,0
4,Целлюлоза,1,0.0,4,3,0,0,0,19,0,...,8,0,323,8,6,0,0,0,0,0


In [58]:
df_structed_transpose.isna().sum()[lambda x: x>0]

Arthrobacter    1
dtype: int64

### 2.3 Визуализация наблюдений

#### 2.3.1 Попарное сравнение по группам внутри таргета разница между 1 и 2

In [63]:
def plot_diff_groups():
    for ind, row in df_structed_transpose[['target']].drop_duplicates().reset_index(drop=True).iterrows():
        df_target = df_structed_transpose[df_structed_transpose['target'] == row['target']]
        df_group_1 = df_target[df_target['group'] == 1]
        df_group_2 = df_target[df_target['group'] == 2]
plot_diff_groups()